In [1]:
soil_properties = pd.read_csv("/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/soilProperties20222023.csv")
soil_properties.info()
soil_properties[['x', 'y']]= soil_properties['sample'].str.split(' ', n = 1 , expand = True)
soil_properties['plot'] = soil_properties['site'].astype(str) + '_' + soil_properties['plot'].astype(str)
soil_properties.drop(columns = {'sample', 'site'}, inplace = True)
#pd.set_option('display.max_rows', None) データを全て確認できる
#pd.set_option('display.max_columns', None)
soil_properties.head()
#seriesのデータ型を文字列に変更するのがastype(str)
#strは文字列から数値 or 数値から文字列に変換する関数

mean_soiltemp = pd.read_csv("/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/meanSoilTempDf20160723.csv")
mean_soiltemp.info()
mean_soiltemp.head()

qPCRdf = pd.merge(soil_properties, mean_soiltemp, on = 'plot', how = 'inner')
qPCRdf['all_info'] = qPCRdf[['plot','x','y','depth','season']].astype(str).agg('-'.join, axis = 1)
qPCRdf.drop(columns = {'plot','x','y','depth','season'}, inplace = True)
qPCRdf = qPCRdf.reset_index(drop = True)
qPCRdf.head()
#　agg関数を使用するとinndexになるのでreset_index()を使用してデータフレームの列名に直す

fungi_bacteria_copynum = pd.read_csv('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/fungiBacteriaCopyNum.csv')
fungi_bacteria_copynum.info()
fungi_bacteria_copynum.drop(columns = ['serial'], inplace = True)
fungi_bacteria_copynum.rename(columns = {'subplot_x':'x', 'subplot_y':'y'}, inplace = True)
fungi_bacteria_copynum['all_info'] = fungi_bacteria_copynum[['plot','x','y','depth','season']].astype(str).agg('-'.join, axis = 1)
fungi_bacteria_copynum.drop(columns = {'plot','x','y','depth','season'}, inplace = True)
fungi_bacteria_copynum = fungi_bacteria_copynum.copy()
fungi_bacteria_copynum.head()
# copy数が '0'の場所をNAに直す
# print(qPCRdf.columns)
# print(fungi_bacteria_copynum.columns)

qPCRdf = pd.merge(qPCRdf,fungi_bacteria_copynum, on = 'all_info', how = 'inner' )
qPCRdf[['plot','x','y','depth','season']] = qPCRdf['all_info'].str.split('-' , n = 5, expand = True)
qPCRdf.drop(columns =  {'all_info'}, inplace = True)
qPCRdf.head()

qPCRdf['all_info'] = qPCRdf[['plot','x','y']].astype(str).agg('-'.join, axis = 1)
qPCRdf.drop(columns = {'plot', 'x', 'y' }, inplace = True)
qPCRdf = qPCRdf.reset_index(drop = True)
qPCRdf.head()

under_vagetation = pd.read_csv('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/understoryVegatation2020.csv',
                               encoding ='cp932', usecols =['plot','totalCover2020','bambooCover2020','coordinates'])
under_vagetation.info()
under_vagetation['coordinates'] = under_vagetation['coordinates'].str.replace(r'[()]', '', regex = True)
under_vagetation[['x','y']] = under_vagetation['coordinates'].str.split(',', expand = True)
under_vagetation['all_info'] = under_vagetation[['plot','x','y']].astype(str).agg('-'.join, axis = 1)
under_vagetation.drop(columns = {'coordinates','plot', 'x', 'y'} , inplace = True)
under_vagetation.head()

qPCRdf = pd.merge(qPCRdf, under_vagetation, on = 'all_info', how = 'inner')
qPCRdf[['plot', 'x', 'y']] = qPCRdf['all_info'].str.split('-' , n = 3, expand = True )
qPCRdf.head()

! pip install numpy
import numpy as np
topographicProperties = pd.read_csv('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/topographicPropertiesForSubplot.csv')
topographicProperties.info()
topographicProperties.drop(columns ={'subplot'}, inplace = True)
topographicProperties['asp1']= np.tan(topographicProperties['slope']) * np.cos(topographicProperties['aspect'])
topographicProperties['asp2'] = np.tan(topographicProperties['slope']) * np.sin(topographicProperties['aspect'])
topographicProperties['all_info'] = topographicProperties[['plot', 'x', 'y']].astype(str).agg('-'.join, axis = 1)
topographicProperties.drop(columns = {'plot', 'x', 'y'}, inplace = True)
topographicProperties

qPCRdf = pd.merge(qPCRdf, topographicProperties, on = 'all_info', how = 'inner')
qPCRdf

lightintensity = pd.read_csv ('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/lightIntensity2023.csv')
lightintensity.info()
lightintensity[['x', 'y']] = lightintensity['quadrat'].str.split(',' , expand = True )
lightintensity['tsf'] = lightintensity['TSF_20240717']/ 154
lightintensity.drop(columns ={'DSF_20240717','ISF_20240717','TSF_20240717','quadrat'}, inplace = True)
lightintensity['all_info'] = lightintensity[['plot', 'x', 'y']].astype(str).agg( '-'.join , axis = 1 )
lightintensity.rename(columns ={'WCO_20240717': 'wco'}, inplace = True)
lightintensity.drop(columns = {'plot', 'x', 'y'}, inplace = True)
lightintensity

qPCRdf = pd.merge(qPCRdf, lightintensity, on = 'all_info', how = 'inner')
qPCRdf

litter = pd.read_csv('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/litter20222023.csv')
litter.info()
litter.drop(columns = {'note'}, inplace = True)
litter.rename(columns = {'subplot_x' : 'x', 'subplot_y' : 'y'}, inplace = True) 
litter['all_info'] = litter[['plot','x','y','season']].astype(str).agg('-'.join, axis = 1)
litter = litter.pivot(index = 'all_info', columns = 'category', values = 'weightLitter')
litter = litter.reset_index()
litter[['plot','x', 'y', 'season']] = litter['all_info'].str.split('-', n = 5, expand = True)
litter.drop(columns= {'all_info','soil'}, inplace = True)
litter['all_info'] = litter[['plot', 'x', 'y', 'season']].astype(str).agg('-'.join, axis = 1)
litter.drop(columns = {'plot', 'x', 'y', 'season'}, inplace = True )
litter

qPCRdf[['plot', 'x', 'y']] = qPCRdf['all_info'].str.split('-', n = 3, expand = True)
qPCRdf['all_info'] = qPCRdf[['plot', 'x', 'y', 'season']].astype(str).agg('-'.join, axis = 1)
qPCRdf = qPCRdf.reset_index(drop = True)
qPCRdf = pd.merge(qPCRdf, litter, on = 'all_info', how = 'inner')
qPCRdf.drop(columns = {'all_info'}, inplace = True)
qPCRdf['fungiCopyNum'].replace({'O':pd.NA, 0: pd.NA}, inplace= True)
qPCRdf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      240 non-null    object 
 1   plot      240 non-null    object 
 2   sample    240 non-null    object 
 3   depth     240 non-null    object 
 4   season    240 non-null    object 
 5   temp      240 non-null    float64
 6   hardness  240 non-null    float64
 7   EC        240 non-null    float64
 8   EC2       240 non-null    float64
 9   pH        240 non-null    float64
 10  C/N       240 non-null    float64
dtypes: float64(6), object(5)
memory usage: 20.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   plot          59 non-null     object 
 1   meanSoilTemp  59 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB
<class 'p

/var/folders/2n/k04q38yx087g0pz_q6w7gfd40000gn/T/ipykernel_7173/4092435809.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  qPCRdf['fungiCopyNum'].replace({'O':pd.NA, 0: pd.NA}, inplace= True)


,temp,hardness,EC,EC2,pH,C/N,meanSoilTemp,fungiCopyNum,bacteriaCopyNum,depth,...,topographicWetnessIndex,normalizedHeight,asp1,asp2,wco,tsf,F,H,L,root
0,6.266667,6.733333,0.023333,6.240,4.90,14.13,17.091954,799275896,5.400098e+11,0~10,...,5.857820,0.829627,-0.431510,-0.054426,10.06,12.780000,18.6,32.4,23.6,0.1
1,8.433333,10.833333,0.013333,1.960,5.47,16.55,17.091954,494411415,2.454023e+11,10~20,...,5.857820,0.829627,-0.431510,-0.054426,10.06,12.780000,18.6,32.4,23.6,0.1
2,5.966667,11.000000,0.013333,7.340,5.10,13.15,17.091954,2193772116,5.250106e+11,0~10,...,6.634809,0.800156,-0.353152,0.000537,9.51,9.446558,50.3,175.9,39.0,1.9
3,8.433333,8.000000,0.003333,1.640,5.47,13.96,17.091954,549395139,1.760672e+11,10~20,...,6.634809,0.800156,-0.353152,0.000537,9.51,9.446558,50.3,175.9,39.0,1.9
4,6.800000,11.633333,0.020000,2.610,5.72,13.50,17.091954,3320234279,7.137855e+11,0~10,...,6.181860,0.754798,-0.311618,-0.004503,14.63,17.348766,62.8,227.4,28.9,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,9.100000,5.180000,0.010000,3.400,4.61,11.33,16.194325,1998254267,5.003325e+11,10~20,...,4.784337,0.698345,-0.284045,0.320275,10.11,15.697662,11.2,18.3,5.0,0.1
188,8.600000,6.480000,0.010000,0.772,4.74,10.32,16.194325,7957586915,1.201374e+12,0~10,...,4.846597,0.745629,-0.179899,0.295390,11.00,13.224351,11.7,31.8,32.4,0.4
189,9.400000,6.980000,0.000000,6.450,4.63,12.39,16.194325,490881191,1.367218e+11,10~20,...,4.846597,0.745629,-0.179899,0.295390,11.00,13.224351,11.7,31.8,32.4,0.4
190,8.400000,4.540000,0.010000,2.150,4.62,11.11,16.194325,1926080237,1.821542e+11,0~10,...,5.050172,0.668143,-0.253618,0.326972,7.62,7.261623,27.4,66.4,24.2,0.0


In [2]:
import math
tree = pd.read_csv('/Users/shimizushota/Library/CloudStorage/OneDrive-千葉大学/data/trees.csv', 
                   encoding = 'cp932', usecols = {'plot','X','Y','wamei','GBH_20-21','dead_20-21', 
                                                'dead_16-17', 'GBH_16-17' ,'GBH_12-13'})
tree.info()
tree.rename(columns = {'X' :'x', 'Y': 'y'}, inplace = True )
tree

conifer = tree[ ~ (tree['plot'] == 'NaN' ) & (tree['dead_20-21'] == 0 ) & 
        (tree['wamei'].isin(["カラマツ","ウラジロモミ","ツガ","針葉樹","コメツガ",
                             "トウヒ","ヒメコマツ","モミ","ストローブマツ"])) ]
conifer.info()
coniferBa = conifer[['plot','GBH_20-21', 'dead_20-21' ]]
coniferBa = coniferBa.copy()
coniferBa['coniferBa'] = coniferBa['GBH_20-21']** 2/ math.pi /4
coniferBa = coniferBa.groupby('plot', as_index = False).sum()
coniferBa['coniferBa'] = coniferBa['coniferBa']/ 1000000 / 0.09
coniferBa.drop(columns = {'GBH_20-21', 'dead_20-21'}, inplace =True)
coniferBa

hardwood = tree[ ~ (tree['plot'] == 'NaN') & (tree['dead_20-21'] == 0 ) &
           ( ~ tree ['wamei'].isin (["カラマツ","ウラジロモミ","ツガ","針葉樹","コメツガ",
                             "トウヒ","ヒメコマツ","モミ","ストローブマツ"])) ]
hardwoodBa = hardwood[['plot','GBH_20-21', 'dead_20-21' ]]
hardwoodBa = hardwoodBa.copy()
hardwoodBa['hardwoodBa'] = hardwoodBa['GBH_20-21']** 2/ math.pi /4
hardwoodBa = hardwoodBa.groupby('plot', as_index = False).sum()
hardwoodBa['hardwoodBa'] = hardwoodBa['hardwoodBa']/ 1000000 / 0.09
hardwoodBa.drop(columns = {'GBH_20-21', 'dead_20-21'}, inplace =True)
hardwoodBa

ecm = tree[ ~ (tree['plot'] == 'NaN') & (tree['dead_20-21'] == 0) &
       (tree['wamei'].isin([ "カラマツ" , "ミズナラ" , "ブナ" , "ウダイカンバ" , "イヌブナ" , "ダケカンバ" , "ミズメ" , "クリ" ,
                            "ウラジロモミ" , "シラカンバ" , "クマシデ" , "ツガ" , "アカシデ" , "サワシバ" , "イヌシデ" , "ヤシャブシ" ,
                            "コメツガ" , "ネコシデ" , "トウヒ" , "ヒメコマツ" , "ジゾウカンバ" , "モミ" , "カバノキ類" , "オノオレカンバ" ,
                            "ブナ類" , "バッコヤナギ" , "ストローブマツ" , "アサダ" ]))]
ecmBa = ecm[['plot','GBH_20-21', 'dead_20-21' ]]
ecmBa = ecmBa.copy()
ecmBa['ecmBa'] = ecmBa['GBH_20-21']** 2/ math.pi /4
ecmBa = ecmBa.groupby('plot', as_index = False).sum()
ecmBa['ecmBa'] = ecmBa['ecmBa']/ 1000000 / 0.09
ecmBa.drop(columns = {'GBH_20-21', 'dead_20-21'}, inplace =True)
ecmBa

am = tree[ ~ (tree['plot'] == 'NaN') & (tree['dead_20-21'] == 0) &
       ( ~ tree['wamei'].isin([ "カラマツ" , "ミズナラ" , "ブナ" , "ウダイカンバ" , "イヌブナ" , "ダケカンバ" , "ミズメ" , "クリ" ,
                            "ウラジロモミ" , "シラカンバ" , "クマシデ" , "ツガ" , "アカシデ" , "サワシバ" , "イヌシデ" , "ヤシャブシ" ,
                            "コメツガ" , "ネコシデ" , "トウヒ" , "ヒメコマツ" , "ジゾウカンバ" , "モミ" , "カバノキ類" , "オノオレカンバ" ,
                            "ブナ類" , "バッコヤナギ" , "ストローブマツ" , "アサダ" ]))]
amBa = am[['plot','GBH_20-21', 'dead_20-21' ]]
amBa = amBa.copy()
amBa['amBa'] = amBa['GBH_20-21']** 2/ math.pi /4
amBa = amBa.groupby('plot', as_index = False).sum()
amBa['amBa'] = amBa['amBa']/ 1000000 / 0.09
amBa.drop(columns = {'GBH_20-21', 'dead_20-21'}, inplace =True)
amBa

deadBa = tree[ ( tree['dead_16-17']  == 1 ) | ( tree['dead_20-21'] == 1 ) ]
deadBa = deadBa.copy()
deadBa['deadGBH'] = np.where( deadBa['dead_16-17'] == 1 , deadBa['GBH_12-13'],
                          np.where(deadBa['dead_20-21'] == 1, deadBa['GBH_16-17'] , 'NA'))
deadBa = deadBa[ ~ (deadBa['deadGBH'] == 'NaN') & ~ (deadBa['deadGBH'] == 'nan' ) ]
deadBa.drop( columns = {'wamei', 'GBH_12-13', 'GBH_16-17', 'dead_16-17', 'GBH_20-21', 'dead_20-21', 'x', 'y' }, inplace = True)
deadBa['deadGBH'] = pd.to_numeric(deadBa['deadGBH'], errors='coerce') #deadBa全体をseries(一列)にしてしまった
deadBa['deadBa'] = deadBa['deadGBH']** 2/ math.pi /4 
deadBa = deadBa.groupby('plot', as_index = False).sum()
deadBa['deadBa'] = deadBa['deadBa']/ 1000000 / 0.09
deadBa

baDf = pd.merge(coniferBa, hardwoodBa, on = 'plot', how = 'inner' )
baDf = pd.merge( baDf, ecmBa , on = 'plot', how = 'inner' )
baDf = pd.merge( baDf, amBa, on = 'plot', how = 'inner' )
baDf = pd.merge( baDf, deadBa, on = 'plot', how = 'inner' )
baDf['totalBa'] = baDf['ecmBa'] + baDf['amBa']
baDf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   plot        8993 non-null   object 
 1   X           8992 non-null   float64
 2   Y           8992 non-null   float64
 3   wamei       8991 non-null   object 
 4   GBH_12-13   8291 non-null   float64
 5   GBH_16-17   8618 non-null   float64
 6   dead_16-17  8850 non-null   float64
 7   GBH_20-21   7321 non-null   float64
 8   dead_20-21  8993 non-null   float64
dtypes: float64(7), object(2)
memory usage: 633.4+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1755 entries, 0 to 8913
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   plot        1755 non-null   object 
 1   x           1755 non-null   float64
 2   y           1755 non-null   float64
 3   wamei       1755 non-null   object 
 4   GBH_12-13   1694 non-null   float64
 

,plot,coniferBa,hardwoodBa,ecmBa,amBa,deadGBH,deadBa,totalBa
0,I01_F,12.836480,21.413227,27.823411,6.426296,11387.0,3.718821,34.249707
1,I01_N,10.810904,27.757481,26.313080,12.255305,10579.0,2.732907,38.568384
2,I02_F,29.405057,18.352795,40.158271,7.599580,7459.0,5.376550,47.757852
3,I02_N,9.303941,33.206906,24.523602,17.987244,7708.0,4.723285,42.510847
4,I03_N,6.662425,26.864757,13.631702,19.895479,6282.0,2.075865,33.527182
5,I04_F,15.298803,28.684343,26.655934,17.327213,14501.0,6.488925,43.983147
6,I04_N,8.922887,28.735917,23.902119,13.756684,12479.0,3.561878,37.658803
7,I05_F,7.665690,36.192383,29.318001,14.540072,9871.0,3.458737,43.858073
8,I05_N,11.764948,29.833732,30.162791,11.435889,11896.0,5.541236,41.598680
9,I06_F,8.519798,28.248929,24.342865,12.425861,6515.0,2.783985,36.768727
